# Example on how to run BBA

Code particular for SIRIUS machine is commented.

Remember to appropriately setup the the response matrix and the reference orbit for SOFB and the center orbit around which the BPM scans will be made.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyacal
from pyacal.experiments.bba import BBA

# from siriuspy.clientconfigdb import ConfigDBClient

In [2]:
pyacal.set_facility('sirius')

In [3]:
# reference orbit for correction and position scan is required

# client = ConfigDBClient(config_type='si_orbit')
# orb = client.get_config_value('ref_orb')

# reforbx = np.array(orb["x"]) * 1e3  # [um] -> [nm]
# reforby = np.array(orb["y"]) * 1e3  # [um] -> [nm]

In [4]:
# orbit response matrix to correct and scan position is required

# client = ConfigDBClient(config_type='si_orbcorr_respm')
# respm = np.array(client.get_config_value('ref_respmat'))
# respm *= 1e3 # [um/urad] -> [nm/urad]

In [3]:
bba = BBA(accelerator='SI', isonline=True)
sofb = bba.devices["sofb"]

In [4]:
bba.params.deltaorbx = 100_000  # use value with appropriated units here
bba.params.deltaorby = 100_000  # use value with appropriated units here
bba.params.meas_nrsteps = 8
bba.params.quad_delta_stren = 0.020  # use value with appropriated units here
bba.params.quad_max_stren = 100.000  # use value with appropriated units here
bba.params.quad_min_stren = -100.000  # use value with appropriated units here
bba.params.quad_nrcycles = 1
bba.params.wait_correctors = 0.300  # [s]
bba.params.wait_quadrupole = 0.300  # [s]
bba.params.timeout_wait_orbit = 3.000  # [s]
bba.params.sofb_nrpoints = 20
bba.params.sofb_maxcorriter = 5
bba.params.sofb_maxorberr = 5_000  # use value with appropriated units here

bba.bpms2dobba = bba.data["bpmnames"][:1]

# set reference orbit as center of position scans
bba.data["scancenterx"] = np.zeros(len(bba.data['bpmnames']))
bba.data["scancentery"] = np.zeros(len(bba.data['bpmnames']))

# set reference orbit for orbit correction
sofb.ref_orbx = np.zeros(len(bba.data['bpmnames']))
sofb.ref_orby = np.zeros(len(bba.data['bpmnames']))

# set orbit response matrix
shape = (2*sofb.nr_bpms, sofb.nr_cors)
dummy_mat = np.zeros(shape)
min_sz = min(*shape)
dummy_mat[:min_sz, :min_sz] = np.eye(min_sz)
sofb.respmat = dummy_mat

print(bba)

Params
    deltaorbx                = 100000.000  
    deltaorby                = 100000.000  
    meas_nrsteps             =         8  
    quad_delta_stren         =     0.020  
    quad_max_stren           =   100.000  
    quad_min_stren           =  -100.000  
    quad_nrcycles            =     1.000  
    wait_correctors          =     0.300  [s]
    wait_quadrupole          =     0.300  [s]
    timeout_wait_orbit       =     3.000  [s] (get orbit)
    sofb_nrpoints            =        20  
    sofb_maxcorriter         =         5  
    sofb_maxorberr           =  5000.000  
    
Connected?  True

             BPM                  Quad         Xc     Yc    
000:       BPM-01M2             QS-01M2         0.0    0.0  



In [7]:
bba.start()

Starting measurement at 2024-07-01 23h38m36s

Correcting Orbit...Ok!

001/001
23h38m51s --> Doing BBA for BPM 000: BPM01M2
cycling QS01M2: . Ok!
    01/09 --> orbit corr: 

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.6/site-packages/epics/ca.py", line 2016, in run
    threading.Thread.run(self)
  File "/usr/local/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/site-packages/pyacal/experiments/base.py", line 201, in _run
    self._target()
  File "/usr/local/lib/python3.6/site-packages/pyacal/experiments/bba.py", line 1046, in _meas_bba
    self._meas_bba_single_bpm(bpm)
  File "/usr/local/lib/python3.6/site-packages/pyacal/experiments/bba.py", line 1117, in _meas_bba_single_bpm
    bpmname, x0+dorbsx[i], y0+dorbsy[i])
  File "/usr/local/lib/python3.6/site-packages/pyacal/experiments/bba.py", line 155, in correct_orbit_at_bpm
    residue=self.params.sofb_maxorberr)
TypeError: 'NoneType' object is not iterable



In [8]:
bba.stop()